In [1]:
import time
import os
import copy

import matplotlib.pyplot as plt
import numpy as np
import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms

import settings

%matplotlib inline

In [2]:
from vocabulary_utils import build_word2vec

In [3]:
from gensim.models import KeyedVectors

In [4]:
model, num_layers = build_word2vec('dataset1/annotations/train.json')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [5]:
print(num_layers)

20


In [6]:
model.wv.save("models_weights/wordvectors.kv")

In [7]:
wv = KeyedVectors.load("models_weights/wordvectors.kv", mmap='r')
num_layers = 20

In [8]:
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [10]:
from CocoDataset import CocoDataset, collate_fn

In [11]:
images_dir = 'dataset1/images/val'
ann_filename = 'dataset1/annotations/val.json'
dataset_val = CocoDataset(images_dir, ann_filename, wv, transform=data_transforms)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [12]:
images_dir = 'dataset1/images/train'
ann_filename = 'dataset1/annotations/train.json'
dataset_train = CocoDataset(images_dir, ann_filename, wv, transform=data_transforms)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
data_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=2,
                                             shuffle=True, num_workers=1, collate_fn=collate_fn)
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=2,
                                             shuffle=True, num_workers=1, collate_fn=collate_fn)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
dataloaders = {'train': data_loader_train, 'val': data_loader_val}

In [ ]:
# so you need to figure out embed_size, hidden_size, vocab, num_layers
# embed_size = aprox 100
# hidden_size = same as embed_dim
# num_layers is the max length among all sequences

In [15]:
num_embeddings = len(wv.vectors)
embed_size = settings.EMBED_SIZE
hidden_size = settings.EMBED_SIZE

In [16]:
from EncoderCNN import EncoderCNN
from DecoderRNN import DecoderRNN

In [17]:
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, wv.vectors, hidden_size, num_embeddings, num_layers).to(device)

In [18]:
learning_rate = 0.001

In [19]:
# Loss and optimizer
loss_function = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(params, lr=learning_rate)
# for now lets try default step_size and gamma
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2)

In [20]:
from ModelTrainer import ModelTrainer

In [21]:
path = 'models_weights'# you have to use custom dataset and custom collate-fn

In [22]:
trainer = ModelTrainer(encoder, decoder, optimizer, scheduler, loss_function, dataloaders, path)

In [23]:
encoder, decoder, losses, accuracies = trainer.train(verbose=True)

Epoch 0/9
----------


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/Applications/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Applications/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/admin/Documents/image_captioning/CocoDataset.py", line 52, in __getitem__
    target = torch.Tensor(caption)
TypeError: must be real number, not Vocab
